# Constants & Libs

In [1]:
from pathlib import Path
import os
from tqdm import tqdm

In [2]:
os.getcwd()

'D:\\PythonProjects\\AML_test_task\\exploration\\notebooks'

In [3]:
PATH_TO_PROJECT = Path(os.getcwd()).parent.parent

In [4]:
os.chdir(PATH_TO_PROJECT)

In [5]:
PATH_TO_PROJECT

WindowsPath('D:/PythonProjects/AML_test_task')

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
import numpy as np
import re

from exploration.Modules.CheckerClass import Checker
from exploration.Modules.VisualisationClass import Vizualization

from feature_engine.encoding import (OneHotEncoder,
                                     CountFrequencyEncoder,
                                     OrdinalEncoder, MeanEncoder,
                                     DecisionTreeEncoder)

from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from feature_engine.imputation import ArbitraryNumberImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from matplotlib import pyplot

In [8]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [9]:
from sklearn.ensemble import IsolationForest

In [10]:
from feature_engine.imputation import AddMissingIndicator

In [11]:
from sklearn.model_selection import GridSearchCV
import joblib

In [12]:
PATH_TO_TRAIN_DATA = PATH_TO_PROJECT.joinpath('data/raw_data/train.csv')
PATH_TO_TEST_DATA = PATH_TO_PROJECT.joinpath('data/raw_data/test.csv')

In [13]:
PATH_TO_ARTEFACTS = PATH_TO_PROJECT.joinpath('exploration/artefacts')

In [14]:
PATH_TO_PREPARED_DATA = PATH_TO_PROJECT.joinpath('data/prepared_data')

# Read file

In [15]:
train_data = pd.read_csv(PATH_TO_TRAIN_DATA)

In [16]:
test_data = pd.read_csv(PATH_TO_TEST_DATA)

# EDA Train data

In [18]:
train_data

,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,...,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,entity_0017_cd,target
0,0,711 days,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,...,NaN,1.000000,3,446.0,207,1.166190,NaN,1.008388,d,1
1,1,218 days,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,...,0.975221,0.996865,3,330.0,200,1.753914,NaN,1.001191,d,1
2,2,462 days,-0.800151,2.204143,-0.514011,0.961015,-0.116245,0.218591,1.152742,0.955213,...,NaN,0.781602,2,191.0,82,1.548657,NaN,0.989861,c,1
3,3,13 days,-1.543821,2.044538,-1.288806,0.958519,-0.485421,0.534662,0.498838,0.913363,...,NaN,0.000000,4,737.0,328,1.385834,NaN,0.998892,a,1
4,4,371 days,-1.564026,2.291450,-1.680573,1.553684,-0.931364,0.613911,1.167507,0.668621,...,NaN,1.000000,1,64.0,43,2.250021,NaN,0.993448,d,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106230,104358,897 days,-1.205612,2.327263,-1.457181,1.016395,-0.906506,0.233363,0.963181,0.827734,...,1.000000,1.000000,1,NaN,33,NaN,NaN,1.005675,d,1
106231,104359,901 days,-0.927309,1.813716,-1.121229,0.124153,-0.361653,0.083467,0.776452,0.444127,...,0.000000,0.000000,3,NaN,224,NaN,NaN,1.004245,a,1
106232,104360,902 days,-0.741633,2.044089,-1.402247,1.136629,-0.324541,0.191690,0.954457,0.785869,...,NaN,1.000000,1,NaN,122,NaN,NaN,1.000972,d,1
106233,104361,902 days,-1.824886,1.890309,-1.443270,1.245961,-0.621427,1.198131,1.469549,0.379104,...,NaN,NaN,1,NaN,225,NaN,NaN,1.011883,a,1


In [19]:
test_data

,Id,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,...,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,entity_0017_cd
0,0,104363,552 days,-1.786346,2.074665,-1.504628,1.143300,-0.979666,0.750931,1.091962,...,NaN,1.0,0.997459,3,339.0,173,2.300125,NaN,-0.003703,d
1,1,104364,152 days,-1.555227,1.703838,-1.682477,1.143936,-1.363388,0.921123,1.209163,...,NaN,1.0,NaN,1,98.0,54,1.598273,NaN,-0.009720,a
2,2,104365,714 days,-1.350453,1.907620,-1.563653,1.271580,-0.902961,0.582773,1.127565,...,NaN,1.0,1.000000,2,235.0,109,1.368001,NaN,-0.011771,d
3,3,104366,5 days,-1.757556,2.429529,-1.438695,0.913385,0.016773,0.629686,1.096519,...,NaN,NaN,NaN,1,92.0,19,2.023099,NaN,0.009288,a
4,4,104367,720 days,-1.466042,1.946119,-1.857461,1.244526,-0.474639,0.481864,0.748999,...,NaN,1.0,1.000000,2,221.0,125,1.388213,NaN,0.002893,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26819,26819,130385,985 days,-0.475784,1.830841,-0.879027,0.850800,-0.713023,-0.083066,0.878643,...,NaN,1.0,NaN,1,NaN,150,NaN,NaN,0.000116,a
26820,26820,130386,989 days,-1.165567,2.230850,-1.497002,1.218585,-0.691802,0.125346,0.572959,...,NaN,NaN,1.000000,1,NaN,5,NaN,NaN,0.004168,d
26821,26821,130387,994 days,-1.172932,2.385078,-1.313173,1.037791,-0.495026,0.348923,1.104281,...,NaN,NaN,0.000000,2,NaN,24,NaN,NaN,0.000012,a
26822,26822,130388,992 days,-0.663178,2.037553,-0.905565,0.935210,-0.898176,0.358711,1.022628,...,NaN,0.0,NaN,4,NaN,451,NaN,NaN,0.008410,a


In [25]:
# совпадает ли Id с индексом
sum(test_data['Id'].values != test_data.index.values)

0

In [26]:
# удаление столбца Id
test_data.drop(columns='Id', inplace=True)

## watch types, NaNs and etc.

In [27]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106235 entries, 0 to 106234
Data columns (total 84 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         106235 non-null  int64  
 1   fact_dt         106235 non-null  object 
 2   emb_0000        106235 non-null  float64
 3   emb_0001        106235 non-null  float64
 4   emb_0002        106235 non-null  float64
 5   emb_0003        106235 non-null  float64
 6   emb_0004        106235 non-null  float64
 7   emb_0005        106235 non-null  float64
 8   emb_0006        106235 non-null  float64
 9   emb_0007        106235 non-null  float64
 10  emb_0008        106235 non-null  float64
 11  emb_0009        106235 non-null  float64
 12  emb_0010        106235 non-null  float64
 13  emb_0011        106235 non-null  float64
 14  emb_0012        106235 non-null  float64
 15  emb_0013        106235 non-null  float64
 16  emb_0014        106235 non-null  float64
 17  emb_0015  

In [28]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26824 entries, 0 to 26823
Data columns (total 83 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         26824 non-null  int64  
 1   fact_dt         26824 non-null  object 
 2   emb_0000        26824 non-null  float64
 3   emb_0001        26824 non-null  float64
 4   emb_0002        26824 non-null  float64
 5   emb_0003        26824 non-null  float64
 6   emb_0004        26824 non-null  float64
 7   emb_0005        26824 non-null  float64
 8   emb_0006        26824 non-null  float64
 9   emb_0007        26824 non-null  float64
 10  emb_0008        26824 non-null  float64
 11  emb_0009        26824 non-null  float64
 12  emb_0010        26824 non-null  float64
 13  emb_0011        26824 non-null  float64
 14  emb_0012        26824 non-null  float64
 15  emb_0013        26824 non-null  float64
 16  emb_0014        26824 non-null  float64
 17  emb_0015        26824 non-null 

### types

In [29]:
for col in train_data.columns:
    print(col)
    print(train_data[col].map(type).unique(), '\n')

user_id
[<class 'int'>] 

fact_dt
[<class 'str'>] 

emb_0000
[<class 'float'>] 

emb_0001
[<class 'float'>] 

emb_0002
[<class 'float'>] 

emb_0003
[<class 'float'>] 

emb_0004
[<class 'float'>] 

emb_0005
[<class 'float'>] 

emb_0006
[<class 'float'>] 

emb_0007
[<class 'float'>] 

emb_0008
[<class 'float'>] 

emb_0009
[<class 'float'>] 

emb_0010
[<class 'float'>] 

emb_0011
[<class 'float'>] 

emb_0012
[<class 'float'>] 

emb_0013
[<class 'float'>] 

emb_0014
[<class 'float'>] 

emb_0015
[<class 'float'>] 

emb_0016
[<class 'float'>] 

emb_0017
[<class 'float'>] 

emb_0018
[<class 'float'>] 

emb_0019
[<class 'float'>] 

emb_0020
[<class 'float'>] 

emb_0021
[<class 'float'>] 

emb_0022
[<class 'float'>] 

emb_0023
[<class 'float'>] 

emb_0024
[<class 'float'>] 

emb_0025
[<class 'float'>] 

emb_0026
[<class 'float'>] 

emb_0027
[<class 'float'>] 

emb_0028
[<class 'float'>] 

emb_0029
[<class 'float'>] 

emb_0030
[<class 'float'>] 

emb_0031
[<class 'float'>] 

emb_0032
[<class 'fl

In [30]:
for col in test_data.columns:
    print(col)
    print(test_data[col].map(type).unique(), '\n')

user_id
[<class 'int'>] 

fact_dt
[<class 'str'>] 

emb_0000
[<class 'float'>] 

emb_0001
[<class 'float'>] 

emb_0002
[<class 'float'>] 

emb_0003
[<class 'float'>] 

emb_0004
[<class 'float'>] 

emb_0005
[<class 'float'>] 

emb_0006
[<class 'float'>] 

emb_0007
[<class 'float'>] 

emb_0008
[<class 'float'>] 

emb_0009
[<class 'float'>] 

emb_0010
[<class 'float'>] 

emb_0011
[<class 'float'>] 

emb_0012
[<class 'float'>] 

emb_0013
[<class 'float'>] 

emb_0014
[<class 'float'>] 

emb_0015
[<class 'float'>] 

emb_0016
[<class 'float'>] 

emb_0017
[<class 'float'>] 

emb_0018
[<class 'float'>] 

emb_0019
[<class 'float'>] 

emb_0020
[<class 'float'>] 

emb_0021
[<class 'float'>] 

emb_0022
[<class 'float'>] 

emb_0023
[<class 'float'>] 

emb_0024
[<class 'float'>] 

emb_0025
[<class 'float'>] 

emb_0026
[<class 'float'>] 

emb_0027
[<class 'float'>] 

emb_0028
[<class 'float'>] 

emb_0029
[<class 'float'>] 

emb_0030
[<class 'float'>] 

emb_0031
[<class 'float'>] 

emb_0032
[<class 'fl

fact_dt только строковый тип, необходимо получить числа

столбцы в train и test в одинаковых форматах

### get days from fact_dt

In [31]:
train_data

,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,entity_0017_cd,target
0,0,711 days,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,NaN,NaN,NaN,1.000000,3,446.0,207,1.166190,NaN,1.008388,d,1
1,1,218 days,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,NaN,NaN,0.975221,0.996865,3,330.0,200,1.753914,NaN,1.001191,d,1
2,2,462 days,-0.800151,2.204143,-0.514011,0.961015,-0.116245,0.218591,1.152742,0.955213,-1.075719,2.451778,-3.058330,2.097131,-2.285532,0.062348,-2.129199,0.829396,0.642969,-1.849109,-0.553088,0.575978,-0.848553,1.657650,1.695016,0.018969,2.155971,-0.469931,-0.967494,-0.895899,-0.364366,0.944774,-0.673376,0.791378,-0.214279,0.588130,0.450273,-0.892531,0.754762,-0.814439,-0.444689,-0.521155,-0.792310,2.015573,-1.363314,0.924737,0.155714,-0.670974,-0.793958,-0.905191,2.167285,-1.523553,-0.857246,1.325879,1.684667,0.385727,0.096711,-2.704222,-1.160303,0.934533,-1.503963,-0.016609,-0.465335,-1.034321,0.653358,-0.727603,4,0,4,0,2,0.987922,NaN,NaN,NaN,0.781602,2,191.0,82,1.548657,NaN,0.989861,c,1
3,3,13 days,-1.543821,2.044538,-1.288806,0.958519,-0.485421,0.534662,0.498838,0.913363,-1.150698,0.682353,-2.342354,1.619433,-2.008181,0.317489,-2.045199,0.819717,0.359040,-1.013714,0.515654,0.962981,-0.895936,2.444055,1.650715,0.488509,2.050253,-0.513660,-1.617719,-0.198799,0.332519,0.498607,-1.393829,0.163533,-0.217686,0.988170,-0.014833,-1.646648,1.749996,0.100091,-0.908938,-1.034925,-1.161342,2.306280,-1.087395,0.695601,1.123252,0.504989,-0.053581,-1.101684,2.047417,-0.872976,-1.436043,1.493698,1.900455,-0.137229,-0.173302,-3.522463,-1.296566,0.875170,-1.407083,0.554178,-0.708667,-1.267630,0.610148,-0.363370,4,4,4,0,2,0.990692,NaN,NaN,NaN,0.000000,4,737.0,328,1.385834,NaN,0.998892,a,1
4,4,371 days,-1.564026,2.291450,-1.680573,1.553684,-0.931364,0.613911,1.167507,0.668621,-0.892333,0.613059,-1.565685,1.777778,-1.983498,0.378075,-2.488828,0.823546,0.677798,-1.135342,1.158549,1.225101,-0.924940,2.248060,1.660758,-0.238992,2.256136,-1.227770,-1.524666,-0.133477,0.356619,0.541877,-1.210004,0.734213,-0.173542,0.475880,0.270290,-

выделяем количество дней из строк в fact_dt

In [32]:
def get_days(data: pd.DataFrame, col: str):
    day_numbers = []
    for amount_days in data[col]:
        days = int(re.search(r'[0-9]+', amount_days)[0])
        day_numbers.append(days)
    data[col] = day_numbers
    return data

In [33]:
train_data = get_days(train_data, 'fact_dt')
test_data = get_days(test_data, 'fact_dt')

### describe features

In [34]:
train_data.describe()

,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,target
count,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,106235.000000,100611.000000,318.000000,25363.000000,53533.000000,80846.000000,106235.000000,100611.000000,106235.000000,100611.000000,25363.000000,106235.000000,106235.000000
mean,52101.275644,466.069007,-1.447150,2.038240,-1.425039,1.069427,-0.804702,0.608138,0.918032,0.703750,-0.975855,1.107644,-1.838451,1.872429,-2.191871,0.112067,-2.278590,0.821131,0.734921,-1.460154,0.519207,0.981252,-0.938018,2.286857,1.671714,0.168731,2.146466,-0.734904,-1.273078,-0.188158,0.215733,0.642495,-1.260757,0.513358,-0.506277,0.464514,0.291281,-0.602659,0.618408,-0.777157,-0.525331,-0.712631,-1.214260,2.234507,-1.429094,0.707615,0.588265,0.196041,-0.187724,-0.882306,2.121601,-1.023574,-1.231388,1.404643,2.016098,-0.127261,0.234212,-3.423351,-1.250541,0.703465,-1.535187,1.105877,-0.501971,-0.866368,0.333920,-0.558501,2.148407,2.257025,2.273601,0.634612,1.021142,0.978599,1.547170,1.259709,0.716707,0.767155,2.459331,457.397054,264.276547,1.929456,1.627647,0.842320,0.842274
std,30099.320678,268.730428,0.363637,0.281269,0.286075,0.251095,0.309413,0.291788,0.253424,0.352944,0.252536,0.505300,0.432134,0.164518,0.247516,0.313158,0.264380,0.008568,0.234689,0.296830,0.486708,0.215993,0.223296,0.247693,0.031940,0.271721,0.111647,0.292978,0.250400,0.360687,0.271154,0.304653,0.272304,0.276338,0.293034,0.309549,0.330108,0.436585,0.390578,0.267849,0.532275,0.399445,0.230552,0.268174,0.280175,0.339950,0.224409,0.382007,0.397642,0.229347,0.223442,0.215570,0.292541,0.127393,0.250046,0.338753,0.303270,0.383455,0.107971,0.299871,0.196661,0.476119,0.188877,0.325132,0.423928,0.226914,1.610670,1.972596,2.875588,0.769521,0.740323,0.020519,1.107463,1.714666,0.437200,0.405137,1.714640,890.794448,561.939743,0.872017,2.036140,0.364647,0.364485
min,0.000000,0.000000,-2.666167,-0.035556,-2.387651,-0.389685,-2.036931,-0.503425,-0.400928,-0.810604,-1.952538,-0.423765,-3.705689,1.110629,-3.472453,-1.440668,-3.265103,0.786286,-0.629051,-2.730158,-1.420660,0.122231,-1.959305,0.972177,1.545092,-1.409719,1.626080,-2.107673,-2.166007,-1.489650,-0.916414,-1.344714,-2.612446,-0.496639,-1.854643,-1.544632,-1.256286,-2.714303,-0.937345,-2.389001,-3.273175,-3.170981,-2.300107,0.927914,-2.494062,

In [36]:
test_data.describe()

,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt
count,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,26824.000000,25272.000000,87.000000,6830.000000,13627.000000,19965.000000,26824.000000,25272.000000,26824.00000,25272.000000,6830.000000,26824.000000
mean,115500.220586,734.677118,-1.450490,2.009285,-1.388864,1.050216,-0.807916,0.624109,0.915447,0.714841,-0.989728,1.195747,-1.851887,1.880268,-2.213637,0.082387,-2.255611,0.819970,0.740121,-1.534341,0.441312,0.943858,-0.931849,2.289847,1.673030,0.177428,2.134491,-0.710192,-1.265271,-0.233795,0.171620,0.680028,-1.268846,0.496559,-0.542333,0.478087,0.312060,-0.485145,0.630404,-0.761635,-0.482351,-0.646236,-1.229236,2.242181,-1.475081,0.706440,0.531626,0.105762,-0.236406,-0.885303,2.156730,-1.025415,-1.222432,1.391540,2.036338,-0.072091,0.241318,-3.346736,-1.238902,0.676322,-1.538079,1.096896,-0.485024,-0.843713,0.377448,-0.549788,2.468424,2.344020,3.229757,0.675664,1.042648,0.977727,1.425287,1.315081,0.661912,0.716559,2.672047,665.681189,389.16448,1.975402,1.638799,-0.000092
std,14407.478810,301.132633,0.368411,0.278591,0.292709,0.248174,0.320981,0.303912,0.252767,0.347377,0.245968,0.497314,0.456981,0.172013,0.243079,0.325432,0.264359,0.008868,0.228988,0.300234,0.490208,0.214414,0.217568,0.264066,0.031894,0.262767,0.110105,0.290211,0.244150,0.373188,0.268338,0.309694,0.283968,0.282167,0.308948,0.322875,0.343152,0.455296,0.386576,0.268037,0.545978,0.411213,0.242733,0.264850,0.275398,0.336426,0.234379,0.390051,0.389724,0.220676,0.222384,0.221086,0.292196,0.127364,0.243918,0.338828,0.310479,0.380019,0.106786,0.292351,0.192200,0.500401,0.191676,0.335685,0.421594,0.221173,3.105877,2.096144,8.173721,0.827253,0.773667,0.020472,0.756863,3.007871,0.457933,0.433678,1.977410,1324.944458,817.48746,0.868520,1.799591,0.010065
min,121.000000,0.000000,-2.611534,0.121522,-2.248160,-0.339003,-1.939328,-0.482909,-0.217089,-0.584591,-1.928606,-0.272455,-3.847916,1.065865,-3.368808,-1.448811,-3.182095,0.788792,-0.869992,-2.694598,-1.260882,0.283757,-1.754549,1.186224,1.553662,-1.263910,1.732875,-1.905180,-2.068264,-1.355775,-0.892932,-0.727244,-2.402370,-0.496211,-1.764886,-1.154950,-1.105481,-2.319793,-0.961277,-2.285734,-2.954007,-2.972020,-2.111563,1.245701,-2.418283,-0.827229,-0.482089,-1.049293,-1.777889,-1.946991,1.307693,-1.757354,-2.575211,1.015696,1.093714,-1.312232,-0.856433,-

макс. значения у entity_0015_ct, entity_0008_ct, entity_0001_ct  разлицаются в несколько раз, возможно выбросы

In [37]:
train_data.groupby('target').agg(['mean', 'min', 'max'])

user_id                 fact_dt           emb_0000            \
                mean min     max        mean min  max      mean       min   
target                                                                      
0       47152.148723   6  104331  568.672416   0  903 -1.541668 -2.666167   
1       53028.058137   0  104362  446.855307   0  903 -1.429450 -2.624755   

                  emb_0001                      emb_0002                      \
             max      mean       min       max      mean       min       max   
target                                                                         
0      -0.237144  2.036990 -0.035556  2.777724 -1.526646 -2.362990  0.099462   
1       0.026171  2.038474  0.040682  2.873151 -1.406012 -2.387651  0.382862   

        emb_0003                      emb_0004                      emb_0005  \
            mean       min       max      mean       min       max      mean   
target                                                                         
0       1.098193 -0.095866  1.941034 -0.776839 -2.036931  0.934639  0.628145   
1       1.064040 -0.389685  2.033400 -0.809920 -1.921724  0.805825  0.604392   

                            emb_0006                      emb_0007            \
             min       max      mean       min       max      mean       min   
target                                                                         
0      -0.440859  1.755637  0.920618 -0.104134  1.825202  0.686231 -0.810604   
1      -0.503425  1.888367  0.917548 -0.400928  1.832388  0.707030 -0.683097   

                  emb_0008                      emb_0009                      \
             max      mean       min       max      mean       min       max   
target                                                                         
0       1.848818 -0.910363 -1.852221 -0.159569  0.942712 -0.423765  2.789981   
1       2.026485 -0.988119 -1.952538 -0.100604  1.138529 -0.164225  2.985820   

        emb_0010                      emb_0011                      emb_0012  \
            mean       min       max      mean       min       max      mean   
target                                                                         
0      -1.820688 -3.398111  0.219723  1.889909  1.131744  3.046282 -2.120579   
1      -1.841777 -3.705689  0.315389  1.869156  1.110629  3.241507 -2.205222   

                           emb_0013                      emb_0014            \
             min      max      mean       min       max      mean       min   
target                                                                        
0      -3.472453 -1.08816  0.136266 -1.060472  1.056048 -2.280529 -3.251984   
1      -3.458290 -0.86904  0.107536 -1.440668  1.265074 -2.278226 -3.265103   

                  emb_0015                      emb_0016                      \
             max      mean       min       max      mean       min       max   
target                                                                         
0      -0.762037  0.822650  0.793881  0.894834  0.723666 -0.629051  1.846910   
1      -0.773088  0.820846  0.786286  0.900031  0.737028 -0.482888  1.805211   

        emb_0017                      emb_0018                      emb_0019  \
            mean       min       max      mean       min       max      mean   
target                                                                         
0      -1.382102 -2.488957 -0.082809  0.692143 -1.219799  2.186388  1.061304   
1      -1.474770 -2.730158  0.228616  0.486823 -1.420660  2.243917  0.966262   

                            emb_0020                      emb_0021            \
             min       max      mean       min       max      mean       min   
target                                                                         
0       0.185832  1.755917 -0.938146 -1.959305 -0.166120  2.313306  0.972177   
1       0.122231  1.751076 -0.937994 -1.935031  0.031838  2.281904  1.132926   

                  emb_0

cтолбец entity_0016_rt полностью разделяет таргет

столбцы с rt видимо должны быть >= 0, но entity_0016_rt < 0, cкорее всего выброс

столбец entity_0007_ct есть только для таргета 1

In [38]:
# сделаем идентификатор, чтобы посмотреть различия в значениях train и test
train_data['train_or_test'] = 'train'
test_data['train_or_test'] = 'test'

In [39]:
pd.concat([train_data, test_data]).groupby('train_or_test').agg(['mean', 'min', 'max'])

user_id                  fact_dt          emb_0000  \
                        mean  min     max        mean min  max     mean   
train_or_test                                                             
test           115500.220586  121  130389  734.677118   0  995 -1.45049   
train           52101.275644    0  104362  466.069007   0  903 -1.44715   

                                   emb_0001                      emb_0002  \
                    min       max      mean       min       max      mean   
train_or_test                                                               
test          -2.611534 -0.040027  2.009285  0.121522  2.829695 -1.388864   
train         -2.666167  0.026171  2.038240 -0.035556  2.873151 -1.425039   

                                   emb_0003                     emb_0004  \
                    min       max      mean       min      max      mean   
train_or_test                                                              
test          -2.248160 -0.160130  1.050216 -0.339003  1.94905 -0.807916   
train         -2.387651  0.382862  1.069427 -0.389685  2.03340 -0.804702   

                                   emb_0005                      emb_0006  \
                    min       max      mean       min       max      mean   
train_or_test                                                               
test          -1.939328  1.033200  0.624109 -0.482909  1.747998  0.915447   
train         -2.036931  0.934639  0.608138 -0.503425  1.888367  0.918032   

                                   emb_0007                      emb_0008  \
                    min       max      mean       min       max      mean   
train_or_test                                                               
test          -0.217089  1.728358  0.714841 -0.584591  1.918906 -0.989728   
train         -0.400928  1.832388  0.703750 -0.810604  2.026485 -0.975855   

                                   emb_0009                     emb_0010  \
                    min       max      mean       min      max      mean   
train_or_test                                                              
test          -1.928606 -0.082661  1.195747 -0.272455  2.93605 -1.851887   
train         -1.952538 -0.100604  1.107644 -0.423765  2.98582 -1.838451   

                                   emb_0011                      emb_0012  \
                    min       max      mean       min       max      mean   
train_or_test                                                               
test          -3.847916  0.206465  1.880268  1.065865  2.981401 -2.213637   
train         -3.705689  0.315389  1.872429  1.110629  3.241507 -2.191871   

                                   emb_0013                      emb_0014  \
                    min       max      mean       min       max      mean   
train_or_test                                                               
test          -3.368808 -1.098098  0.082387 -1.448811  1.080524 -2.255611   
train         -3.472453 -0.869040  0.112067 -1.440668  1.265074 -2.278590   

                                   emb_0015                      emb_0016  \
                    min       max      mean       min       max      mean   
train_or_test                                                               
test          -3.182095 -0.783188  0.819970  0.788792  0.903190  0.740121   
train         -3.265103 -0.762037  0.821131  0.786286  0.900031  0.734921   

                                   emb_0017                      emb_0018  \
                    min       max      mean       min       max      mean   
train_or_test                                                               
test          -0.869992  1.701841 -1.534341 -2.694598  0.167882  0.441312   
train         -0.629051  1.846910 -1.460154 -2.730158  0.228616  0.519207   

                                   emb_0019                      emb_0020  \
                    min       max      mean       min       max      mean   
train_or_test              

cтолбец entity_0016_rt имеет абсолютно разные значение на train и test
возможно, стоит его вообще убрать или как-то преобразовать

часть фичей из эмбедингов могут полностью лежать в области < 0 (например emb_0020) на train, но при этом могут иметь значения > 0 на test, возможно выбросы

In [40]:
Vizualization(train_data).numTargetHist(target='target', 
                                        save_folder=f'{PATH_TO_ARTEFACTS}/train_features_target.png')

In [41]:
Vizualization(pd.concat([train_data, test_data])).numTargetHist(
    target='train_or_test', 
    save_folder=f'{PATH_TO_ARTEFACTS}/train_test_features.png'
)

небольшая доля user_id повторяется в train и test

небольшая доля fact_dt повторяется в train и test, но в целом в test значения больше (возможно там более новые значения)

In [42]:
for col in train_data.columns:
    print(col)
    print(train_data[col].value_counts().iloc[:15], '\n')

user_id
2962     6
66802    5
63335    4
88498    4
77304    4
57862    4
15382    4
16932    3
63714    3
29958    3
46652    3
60267    3
56306    3
86067    3
66970    3
Name: user_id, dtype: int64 

fact_dt
327    673
320    618
218    610
252    569
765    559
867    557
827    438
278    423
820    401
313    376
265    373
306    367
859    363
862    327
736    326
Name: fact_dt, dtype: int64 

emb_0000
-1.348416    4
-1.609083    4
-1.597920    3
-1.542863    3
-1.512492    3
-1.561125    3
-0.652880    3
-1.603621    3
-1.609715    3
-1.366084    3
-1.586940    3
-1.530188    3
-1.474688    3
-1.004395    2
-1.811498    2
Name: emb_0000, dtype: int64 

emb_0001
2.133930    5
2.021978    4
2.167220    4
2.146888    3
2.179343    3
2.323759    3
2.116333    3
2.022372    3
2.254876    3
2.018679    3
2.131730    3
2.210071    3
2.239388    3
2.130896    3
2.242688    3
Name: emb_0001, dtype: int64 

emb_0002
-1.386116    4
-1.434737    4
-1.738346    3
-1.405117    3
-1.596694 

-0.805219    4
-0.805716    4
-0.854636    3
-0.759899    3
-0.867773    3
-0.832250    3
-0.778306    3
-0.802550    3
-0.802810    3
-0.787866    3
-0.766055    3
-0.917965    2
-0.784888    2
-1.031073    2
-0.840312    2
Name: emb_0037, dtype: int64 

emb_0038
-0.878049    4
-0.885800    4
-0.908703    3
-0.852054    3
-0.880367    3
-0.890202    3
-0.700977    3
-0.903524    3
-0.830195    3
-0.838341    2
-0.900756    2
-0.660640    2
-0.891649    2
-0.587377    2
 0.367473    2
Name: emb_0038, dtype: int64 

emb_0039
-0.995391    4
-1.031668    4
-0.705760    3
-0.993697    3
-1.023073    3
-0.872226    3
-0.991749    3
-1.016563    3
-1.057362    3
-1.214166    2
-0.977400    2
-0.818772    2
-0.569187    2
-1.060713    2
-0.876699    2
Name: emb_0039, dtype: int64 

emb_0040
-1.260601    4
-1.059681    4
-1.329509    3
-1.190930    3
-1.344996    3
-1.067897    3
-0.951018    3
-1.264406    3
-1.288249    3
-1.301448    3
-0.806587    3
-1.301369    3
-1.141485    3
-1.114364 

таргет несбалансирован

In [43]:
for col in test_data.columns:
    print(col)
    print(test_data[col].value_counts().iloc[:10], '\n')

user_id
86403     3
113621    3
78436     3
120009    2
119464    2
118031    2
124517    2
112288    2
119604    2
106638    2
Name: user_id, dtype: int64 

fact_dt
958    316
916    308
988    298
972    296
959    278
964    277
924    271
978    255
974    254
986    251
Name: fact_dt, dtype: int64 

emb_0000
-1.755696    2
-1.341290    2
-1.585179    2
-1.889131    2
-1.482621    2
-1.366715    2
-1.381594    2
-1.403103    2
-1.282072    2
-1.809286    2
Name: emb_0000, dtype: int64 

emb_0001
2.172972    2
2.200042    2
2.050608    2
1.878144    2
1.905108    2
1.699945    2
2.185523    2
1.993687    2
1.788179    2
2.312823    2
Name: emb_0001, dtype: int64 

emb_0002
-1.030304    2
-1.572146    2
-1.595067    2
-1.595266    2
-1.375573    2
-1.543662    2
-1.597100    2
-1.477283    2
-1.726204    2
-1.085923    2
Name: emb_0002, dtype: int64 

emb_0003
1.184675    2
1.225877    2
1.073177    2
0.991791    2
1.200937    2
1.040255    2
0.901573    2
1.116282    2
1.004667    2

категориальная переменная одна, категории одинаковые для train и test

#### change values

возможно допущена ошибка в данных entity_0016_rt, необходимо преобразовать, иначе невозможно использовать

In [44]:
train_data['entity_0016_rt'] = np.where(train_data['entity_0016_rt'] < 0.5, 
                                        train_data['entity_0016_rt']+1,
                                        train_data['entity_0016_rt'])

In [45]:
test_data['entity_0016_rt'] = np.where(test_data['entity_0016_rt'] < 0.5, 
                                       test_data['entity_0016_rt']+1,
                                       test_data['entity_0016_rt'])

преобразовываем категориальные переменные

In [46]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [47]:
train_data.shape, test_data.shape

((106235, 88), (26824, 87))

#### drop features

In [48]:
# убираем лишнюю колонку
train_data.drop(columns=['train_or_test_train'], inplace=True)
test_data.drop(columns=['train_or_test_test'], inplace=True)

### check duplicates

In [20]:
# проверка на повторяющиеся строки
len(train_data[train_data.duplicated()]), len(test_data[test_data.duplicated()])

(0, 0)

In [22]:
# проверка на повторяющихся юзеров
train_data[train_data.duplicated(subset='user_id', keep=False)]

,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,...,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,entity_0017_cd,target
12,12,726 days,-1.643533,1.682684,-1.746523,0.963847,-0.328123,0.482034,1.108863,0.821330,...,NaN,NaN,0,296.0,131,1.128863,NaN,-0.003594,a,0
13,12,799 days,-1.416820,1.946490,-1.883106,1.324469,-0.636328,0.385955,0.966425,0.533335,...,NaN,NaN,0,282.0,143,2.018415,NaN,-0.015311,a,0
45,44,762 days,-1.761755,2.012707,-1.492169,1.259235,-0.405148,0.689763,0.951283,0.869306,...,0.0,0.000000,5,335.0,153,2.137517,1.0,0.002483,a,0
46,44,833 days,-1.775203,1.742030,-1.626967,1.354893,-0.533353,0.754441,0.824805,0.665218,...,1.0,0.501799,4,289.0,181,2.057399,NaN,0.994328,b,1
68,66,197 days,-1.841182,2.385974,-1.252883,1.293794,-0.723744,0.498540,0.737070,0.832595,...,1.0,1.000000,3,661.0,290,1.913532,NaN,0.989467,d,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104793,102923,782 days,-1.642335,2.275263,-1.479800,1.068321,-0.842906,0.896496,0.836593,0.965329,...,0.0,0.969697,4,177.0,82,1.876560,1.0,0.993603,d,1
105616,103746,846 days,-1.140465,2.063800,-0.789034,0.946521,-0.491907,0.354838,0.806488,1.142855,...,1.0,0.000000,2,NaN,193,NaN,1.0,0.992997,a,1
105617,103746,881 days,-1.322893,1.449876,-0.949505,0.473813,-0.952306,0.277869,0.543416,1.151305,...,NaN,1.000000,1,201.0,4,2.720551,NaN,1.009941,d,1
105891,104020,854 days,-1.709389,2.396794,-1.495683,1.306259,-0.475099,0.289265,0.467528,1.162600,...,NaN,NaN,0,NaN,4,NaN,NaN,1.012182,a,1


In [23]:
# величина таргета для повторяющихся юзеров
train_data[train_data.duplicated(subset='user_id', keep=False)].mean()['target']

0.80065717415115

In [52]:
test_data[test_data.duplicated(subset='user_id', keep=False)]

,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,entity_0017_cd_a,entity_0017_cd_b,entity_0017_cd_c,entity_0017_cd_d
359,104704,431,-2.066522,1.966859,-1.528340,1.453484,-0.718544,0.911754,1.062410,0.616694,-0.872881,0.933647,-1.928727,1.793289,-2.305893,-0.001024,-2.450168,0.820383,0.760159,-1.621324,0.729584,0.975557,-0.859787,2.239219,1.657698,0.018921,2.278338,-0.679678,-1.525081,-0.118171,0.326690,0.772594,-1.186727,0.615568,-0.922200,0.826500,0.323372,0.192522,-0.029358,-0.680716,-0.288549,-0.128543,-1.277898,2.429869,-1.722851,0.530233,0.398139,0.310824,-0.007137,-0.929553,2.181192,-1.132529,-1.137547,1.390453,2.116252,-0.087726,-0.055409,-3.558435,-1.314607,0.648719,-1.401805,1.084149,-0.482088,-0.945626,0.498697,-0.609320,4,5,4,1,2,0.993627,NaN,NaN,1.000000,0.1847,5,775.0,428,1.169404,NaN,0.979510,1,0,0,0
1204,105513,50,-1.162157,1.716480,-1.567729,1.137163,-0.670023,0.698353,0.996311,0.267187,-1.148728,1.192745,-2.026346,1.879502,-2.263188,-0.514315,-2.409118,0.826884,0.591880,-0.693158,0.742342,0.807234,-0.720370,2.045977,1.680931,0.481274,2.211079,-0.551387,-1.315622,0.283227,-0.019645,0.689687,-1.563434,0.242465,-0.202263,0.563146,0.570932,-1.135687,0.986160,-0.780763,-0.432314,-0.689466,-1.125995,2.840517,-1.423143,-0.045429,0.446980,0.472391,-0.492615,-1.265644,2.680323,-0.890888,-1.440332,1.283876,2.133608,0.277622,-0.228939,-3.651724,-1.418648,0.865692,-1.093523,1.459220,-0.053558,-1.155061,0.137560,-0.410249,1,3,1,0,1,0.972499,NaN,NaN,NaN,1.0000,3,92.0,72,2.439563,NaN,0.991174,0,0,0,1
1205,105513,229,-1.825851,1.951641,-1.335574,0.633140,-0.475159,0.924218,1.037273,0.708950,-0.708111,1.573424,-2.172127,2.489036,-2.404069,-0.513852,-2.273640,0.817807,0.641716,-1.914578,-0.125633,1.072305,-0.883599,2.214850,1.637177,-0.527083,2.000889,-0.605361,-1.223745,-0.096858,0.216297,0.561740,-1.187921,0.864573,0.029613,0.390898,0.110083,0.618313,0.773540,-1.071570,-0.735199,-0.884059,-1.221329,2.496615,-1.146230,0.889324,0.171380,0.384507,-0.417382,-1.148712,2.113016,-0.951908,-1.391111,1.409011,1.966837,-0.246248,0.208078,-3.428563,-1.431747,1.074529,-1.396582,0.733900,-0.409701,-1.087759,0.555046,-0.216816,1,1,1,0,1,0.944772,NaN,NaN,NaN,0.0000,2,29.0,19,3.422229,NaN,1.000307,1,0,0,0
1439,105731,440,-1.482701,2.257778,-1.498943,1.255764,-0.950217,0.709788,1.124223,0.544300,-0.870476,1.301295,-1.725611,1.733603,-2.202664,-0.056708,-2.280956,0.806774,0.732443,-1.701699,0.528079,0.994208,-1.043943,2.166485,1.651007,0.243167,2.195030,-0.781129,-1.389751,-0.328436,0.219821,0.975327,-1.178394,0.598979,-0.821413,0.251362,0.350613,0.308903,0.454298,-0.905390,-0.248947,-0.396719,-1.338874,2.159466,-1.585444,0.679090,0.492235,0.041362,-0.144034,-0.692556,2.247157,-1.067013,-1.070524,1.372688,2.244548,-0.186250,0.232113,-3.752068,-1.225470,0.446036,-1.516925,1.124571,-0.395306,-0.867416,0.201013,-0.713955,2,2,2,0,1,0.996555,NaN,NaN,NaN,1.0000,2,415.0,199,0.846781,NaN,1.012149,0,0,0,1
1440,105731,670,-1.157679,2.531556,-1.286378,1.184968,-0.785515,0.368588,0.939529,0.795854,-1.016653,1.397082,-1.988428,1.776214,-2.139833,0.206772,-2.121300,0.814953,0.725586,-1.598052,0.276505,0.942328,-0.905194,2.322849,1.640967,0.026222,2.146755,-0.537431,-1.250426,-0.554829,0.2

#### generate features

In [53]:
train_data[train_data.duplicated(subset='user_id', keep=False)].mean()

user_id             47614.453998
fact_dt               469.287788
emb_0000               -1.465221
emb_0001                2.028261
emb_0002               -1.445918
                        ...     
target                  0.800657
entity_0017_cd_a        0.420865
entity_0017_cd_b        0.019168
entity_0017_cd_c        0.042716
entity_0017_cd_d        0.517251
Length: 87, dtype: float64

повторяющихся юзеров можно использовать как допю фичу

In [54]:
# определение списка повторяющихся юзеров
duplicated_users_train = train_data[train_data.duplicated(subset='user_id', keep=False)]['user_id'].unique()
duplicated_users_test = test_data[test_data.duplicated(subset='user_id', keep=False)]['user_id'].unique()

In [55]:
# создание доп фичи repeat_user
train_data['repeat_user'] = np.where(train_data['user_id'].isin(duplicated_users_train), 
                                    1, 
                                    0)

test_data['repeat_user'] = np.where(test_data['user_id'].isin(duplicated_users_test), 
                                    1, 
                                    0)

In [56]:
# определение уникальных пользователей в train и test
unique_id_train = set(train_data['user_id'].unique())
unique_id_test = set(test_data['user_id'].unique())                      

In [57]:
# кол-во одинаковых пользователей в train и test
len(unique_id_train & unique_id_test)

679

In [58]:
train_data[train_data['user_id'].isin(unique_id_test)]

,user_id,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,target,entity_0017_cd_a,entity_0017_cd_b,entity_0017_cd_c,entity_0017_cd_d,repeat_user
124,121,350,-1.890744,2.184160,-1.158136,1.001389,-0.438950,0.469440,0.406042,1.107948,-0.811115,0.942303,-2.182426,1.664578,-2.436104,0.456642,-2.351635,0.820554,0.594934,-1.305994,0.193542,0.656115,-0.766771,2.215062,1.691415,0.370130,2.091103,-0.680611,-1.465138,-0.438554,0.478869,1.112768,-0.941595,0.325310,-0.515999,0.146010,0.262213,-0.645548,0.441589,-1.012274,-0.114312,-0.656186,-1.181533,2.363894,-1.155432,1.006066,0.989640,0.052888,-0.426086,-0.806005,2.017655,-1.099682,-1.372062,1.588807,1.866085,-0.004958,0.011010,-3.387224,-1.226592,0.581371,-1.498530,0.308453,-0.443458,-1.015437,0.778363,-0.555450,4,2,3,0,1,0.977620,NaN,NaN,NaN,1.000000,2,128.0,86,2.135881,NaN,0.997093,1,0,0,0,1,0
171,167,257,-1.826668,1.786581,-1.482643,0.805674,-0.816670,0.736682,0.913230,0.318235,-0.976045,1.208800,-1.908009,1.901958,-2.375870,-0.123617,-2.269633,0.819124,0.891083,-1.760172,0.816860,1.046678,-0.955019,2.282848,1.633360,0.270148,2.087589,-0.862906,-1.171192,0.235288,0.176010,0.447668,-1.220212,0.592605,-0.797419,0.301033,-0.088081,-0.431554,0.811673,-0.775111,0.589678,0.032971,-1.858036,2.149906,-1.283504,0.790348,0.633449,0.095291,-0.239463,-1.283039,2.188615,-0.653457,-1.804727,1.421844,2.386932,-0.286264,0.338865,-3.164097,-1.318224,0.842178,-1.485554,1.131333,-0.354346,-1.186358,0.476349,-0.286935,4,5,5,0,1,0.974018,NaN,NaN,NaN,1.000000,4,643.0,420,2.330424,NaN,1.007454,1,0,0,0,1,0
254,250,228,-1.440238,2.214836,-1.616092,1.420996,-0.584427,0.607012,0.861210,0.949874,-0.917273,0.813635,-1.954425,1.876085,-2.023594,0.066285,-2.261424,0.828956,0.528084,-1.117490,0.829449,1.008240,-0.779688,2.251256,1.667732,-0.077130,2.289134,-0.798823,-1.609726,0.055489,0.204845,0.888471,-1.167602,0.540414,-0.188214,0.104168,0.533822,-0.758068,0.332775,-0.808993,-0.747447,-0.809234,-1.133772,2.472788,-1.606191,0.658673,0.686250,0.449803,0.004215,-1.005151,2.037171,-1.172521,-1.010026,1.509619,1.847216,-0.111479,-0.255940,-3.878572,-1.323130,1.004627,-1.482257,0.779462,-0.595427,-1.155467,0.460966,-0.692389,3,4,3,0,1,0.979911,NaN,NaN,NaN,1.000000,3,61.0,28,2.051499,NaN,1.002132,0,0,0,0,1,0
302,296,691,-1.517432,1.987508,-1.294304,1.260496,-0.612171,0.948963,1.318952,0.475409,-1.183762,2.057260,-2.607343,1.634685,-2.426785,-0.307903,-2.284820,0.825964,0.751137,-1.719960,0.299117,0.530970,-0.765148,1.835465,1.641663,-0.087084,2.346842,-0.384433,-1.109227,-0.029254,0.255204,0.976538,-1.006204,0.860553,-0.922937,0.177566,0.392848,0.312427,-0.046892,-0.811332,0.199848,-0.121963,-1.234818,2.650626,-2.036941,0.282154,0.356332,-0.367752,-0.211472,-0.921202,1.968302,-1.031968,-1.043827,1.265769,1.987453,0.339221,-0.101461,-3.109599,-1.293703,0.971127,-1.445572,0.647549,-0.489903,-1.394837,0.784678,-0.530654,8,9,8,4,5,0.965352,NaN,NaN,0.000000,0.593165,13,2798.0,1436,2.695649,NaN,0.996404,0,0,1,0,0,0
309,303,104,-1.033416,2.190748,-1.600735,1.307490,-1.087465,0.590379,1.099830,0.709924,-1.030021,0.824387,-1.615275,1.670905,-1.961193,0.245493,-2.253463,0.817700,0.653061,-1.306183,0.743609,1.225627,-0.983479,2.313111,1.664776,0.196587,2.179994,-0.89400

#### drop features

In [59]:
# удаление признака user_id
train_data.drop(columns='user_id', inplace=True)
test_data.drop(columns='user_id', inplace=True)

### check NaNs

In [60]:
# определение процента пропусков в столбцах train
Checker(train_data).check_missing()

,missing
entity_0006_rt,5.293924
entity_0014_rt,5.293924
entity_0012_ct,5.293924
entity_0010_rt,23.898903
entity_0009_rt,49.608886
entity_0008_ct,76.125571
entity_0015_ct,76.125571
entity_0007_ct,99.700664


In [61]:
# определение процента пропусков в столбцах test
Checker(test_data).check_missing()

,missing
entity_0012_ct,5.785863
entity_0014_rt,5.785863
entity_0006_rt,5.785863
entity_0010_rt,25.570385
entity_0009_rt,49.198479
entity_0015_ct,74.537727
entity_0008_ct,74.537727
entity_0007_ct,99.675664


видно, что пропуски в одних и тех же колонках, при чем у одних и тех же колонках процент пропусков одинаковый, в качестве доп. признаков можно использовать индикаторы пропусков

#### generate new features

In [62]:
ami = AddMissingIndicator()

In [63]:
train_data = ami.fit_transform(train_data)

In [64]:
test_data = ami.fit_transform(test_data)

In [65]:
train_data

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,target,entity_0017_cd_a,entity_0017_cd_b,entity_0017_cd_c,entity_0017_cd_d,repeat_user,entity_0006_rt_na,entity_0007_ct_na,entity_0008_ct_na,entity_0009_rt_na,entity_0010_rt_na,entity_0012_ct_na,entity_0014_rt_na,entity_0015_ct_na
0,711,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,NaN,NaN,NaN,1.000000,3,446.0,207,1.166190,NaN,1.008388,1,0,0,0,1,0,0,1,1,1,0,0,0,1
1,218,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,NaN,NaN,0.975221,0.996865,3,330.0,200,1.753914,NaN,1.001191,1,0,0,0,1,0,0,1,1,0,0,0,0,1
2,462,-0.800151,2.204143,-0.514011,0.961015,-0.116245,0.218591,1.152742,0.955213,-1.075719,2.451778,-3.058330,2.097131,-2.285532,0.062348,-2.129199,0.829396,0.642969,-1.849109,-0.553088,0.575978,-0.848553,1.657650,1.695016,0.018969,2.155971,-0.469931,-0.967494,-0.895899,-0.364366,0.944774,-0.673376,0.791378,-0.214279,0.588130,0.450273,-0.892531,0.754762,-0.814439,-0.444689,-0.521155,-0.792310,2.015573,-1.363314,0.924737,0.155714,-0.670974,-0.793958,-0.905191,2.167285,-1.523553,-0.857246,1.325879,1.684667,0.385727,0.096711,-2.704222,-1.160303,0.934533,-1.503963,-0.016609,-0.465335,-1.034321,0.653358,-0.727603,4,0,4,0,2,0.987922,NaN,NaN,NaN,0.781602,2,191.0,82,1.548657,NaN,0.989861,1,0,0,1,0,0,0,1,1,1,0,0,0,1
3,13,-1.543821,2.044538,-1.288806,0.958519,-0.485421,0.534662,0.498838,0.913363,-1.150698,0.682353,-2.342354,1.619433,-2.008181,0.317489,-2.045199,0.819717,0.359040,-1.013714,0.515654,0.962981,-0.895936,2.444055,1.650715,0.488509,2.050253,-0.513660,-1.617719,-0.198799,0.332519,0.498607,-1.393829,0.163533,-0.217686,0.988170,-0.014833,-1.646648,1.749996,0.100091,-0.908938,-1.034925,-1.161342,2.306280,-1.087395,0.695601,1.123252,0.504989,-0.053581,-1.101684,2.047417,-0.872976,-1.436043,1.493698,1.900455,-0.137229,-0.173302,-3.522463,-1.296566,0.875170,-1.407083,0.554178,-0.708667,-1.267630,0.610148,-0.363370,4,4,4,0,2,0.990692,NaN,NaN,NaN,0.000000,4,737.0,328,1.385834,NaN,0.998892,1,1,0,0,0,0,0,1,1,1,0,0,0,1
4,371,-1.564026,2.291450,-1.680573,1.553684,-0.931364,0.613911,1.167507,0.6

проверим, на одних ли и тех же местах находятся пропуски, если да, то такие фичи одинаковы, часть из них надо удалить

In [66]:
(train_data['entity_0015_ct_na'] == train_data['entity_0008_ct_na']).sum()

106235

In [67]:
(train_data['entity_0006_rt_na'] == train_data['entity_0012_ct_na']).sum()

106235

#### drop features

In [71]:
train_data.drop(columns=['entity_0008_ct_na', 'entity_0012_ct_na', 'entity_0014_rt_na'], inplace=True)
test_data.drop(columns=['entity_0008_ct_na', 'entity_0012_ct_na', 'entity_0014_rt_na'], inplace=True)

### correlations

оценим степень уровень корреляции

In [72]:
corr_data = Checker(train_data).featureCorrelation(target='target', corr_edge=0.7)

In [73]:
corr_data['negativeCorr']

,,coef
entity_0010_rt,entity_0017_cd_a,-0.976036
entity_0006_rt,entity_0014_rt,-0.959860
entity_0017_cd_a,entity_0017_cd_d,-0.912360
emb_0019,emb_0053,-0.837969
entity_0007_ct,entity_0007_ct_na,-0.813172
emb_0004,emb_0059,-0.785750
emb_0009,emb_0045,-0.779329
emb_0004,emb_0010,-0.766618
entity_0004_ct,entity_0009_rt_na,-0.765784
emb_0010,emb_0062,-0.754850


In [74]:
corr_data['positiveCorr']

,,coef
entity_0010_rt,entity_0017_cd_d,0.928030
entity_0012_ct,entity_0013_ct,0.920954
emb_0010,emb_0059,0.863473
emb_0008,emb_0051,0.812598
entity_0002_ct,entity_0011_ct,0.778079
emb_0009,emb_0055,0.768393
entity_0001_ct,entity_0003_ct,0.743085
emb_0008,emb_0046,0.714427
emb_0020,emb_0053,0.704536


### find outliers

In [77]:
train_data.columns

Index(['fact_dt', 'emb_0000', 'emb_0001', 'emb_0002', 'emb_0003', 'emb_0004',
       'emb_0005', 'emb_0006', 'emb_0007', 'emb_0008', 'emb_0009', 'emb_0010',
       'emb_0011', 'emb_0012', 'emb_0013', 'emb_0014', 'emb_0015', 'emb_0016',
       'emb_0017', 'emb_0018', 'emb_0019', 'emb_0020', 'emb_0021', 'emb_0022',
       'emb_0023', 'emb_0024', 'emb_0025', 'emb_0026', 'emb_0027', 'emb_0028',
       'emb_0029', 'emb_0030', 'emb_0031', 'emb_0032', 'emb_0033', 'emb_0034',
       'emb_0035', 'emb_0036', 'emb_0037', 'emb_0038', 'emb_0039', 'emb_0040',
       'emb_0041', 'emb_0042', 'emb_0043', 'emb_0044', 'emb_0045', 'emb_0046',
       'emb_0047', 'emb_0048', 'emb_0049', 'emb_0050', 'emb_0051', 'emb_0052',
       'emb_0053', 'emb_0054', 'emb_0055', 'emb_0056', 'emb_0057', 'emb_0058',
       'emb_0059', 'emb_0060', 'emb_0061', 'emb_0062', 'emb_0063',
       'entity_0001_ct', 'entity_0002_ct', 'entity_0003_ct', 'entity_0004_ct',
       'entity_0005_ct', 'entity_0006_rt', 'entity_0007_ct', 'ent

In [78]:
cols_to_drop=['target', 'entity_0017_cd_a', 'entity_0017_cd_b', 'entity_0017_cd_c',
       'entity_0017_cd_d', 'repeat_user', 'entity_0006_rt_na',
       'entity_0007_ct_na', 'entity_0009_rt_na', 'entity_0010_rt_na',
       'entity_0015_ct_na']

In [79]:
features_train_for_outliers = train_data.drop(columns=cols_to_drop).copy()
features_test_for_outliers = test_data.drop(columns=cols_to_drop[1:]).copy()

In [80]:
features_train_for_outliers.fillna(-9999, inplace=True)
features_test_for_outliers.fillna(-9999, inplace=True)

In [81]:
# cfe = CountFrequencyEncoder(encoding_method='count')

In [82]:
# cfe.fit(features_train_for_outliers)
# features_train_for_outliers = cfe.transform(features_train_for_outliers)
# features_test_for_outliers = cfe.transform(features_test_for_outliers)

In [83]:
features_train_for_outliers

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt
0,711,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,0.0,0.0,-9999.000000,1.000000,3,446.0,207,1.166190,0.0,1.008388
1,218,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,0.0,0.0,0.975221,0.996865,3,330.0,200,1.753914,0.0,1.001191
2,462,-0.800151,2.204143,-0.514011,0.961015,-0.116245,0.218591,1.152742,0.955213,-1.075719,2.451778,-3.058330,2.097131,-2.285532,0.062348,-2.129199,0.829396,0.642969,-1.849109,-0.553088,0.575978,-0.848553,1.657650,1.695016,0.018969,2.155971,-0.469931,-0.967494,-0.895899,-0.364366,0.944774,-0.673376,0.791378,-0.214279,0.588130,0.450273,-0.892531,0.754762,-0.814439,-0.444689,-0.521155,-0.792310,2.015573,-1.363314,0.924737,0.155714,-0.670974,-0.793958,-0.905191,2.167285,-1.523553,-0.857246,1.325879,1.684667,0.385727,0.096711,-2.704222,-1.160303,0.934533,-1.503963,-0.016609,-0.465335,-1.034321,0.653358,-0.727603,4,0,4,0,2,0.987922,0.0,0.0,-9999.000000,0.781602,2,191.0,82,1.548657,0.0,0.989861
3,13,-1.543821,2.044538,-1.288806,0.958519,-0.485421,0.534662,0.498838,0.913363,-1.150698,0.682353,-2.342354,1.619433,-2.008181,0.317489,-2.045199,0.819717,0.359040,-1.013714,0.515654,0.962981,-0.895936,2.444055,1.650715,0.488509,2.050253,-0.513660,-1.617719,-0.198799,0.332519,0.498607,-1.393829,0.163533,-0.217686,0.988170,-0.014833,-1.646648,1.749996,0.100091,-0.908938,-1.034925,-1.161342,2.306280,-1.087395,0.695601,1.123252,0.504989,-0.053581,-1.101684,2.047417,-0.872976,-1.436043,1.493698,1.900455,-0.137229,-0.173302,-3.522463,-1.296566,0.875170,-1.407083,0.554178,-0.708667,-1.267630,0.610148,-0.363370,4,4,4,0,2,0.990692,0.0,0.0,-9999.000000,0.000000,4,737.0,328,1.385834,0.0,0.998892
4,371,-1.564026,2.291450,-1.680573,1.553684,-0.931364,0.613911,1.167507,0.668621,-0.892333,0.613059,-1.565685,1.777778,-1.983498,0.378075,-2.488828,0.823546,0.677798,-1.135342,1.158549,1.225101,-0.924940,2.248060,1.660758,-0.238992,2.256136,-1.227770,-1.524666,-0.133477,0.356619,0.541877,-1.210004,0.734213,-0.173542,0.475880,0.270290,-0.576467,0.275184,-1.012020,-0.616608,-1.166215,-1.109

In [84]:
features_test_for_outliers

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt
0,552,-1.786346,2.074665,-1.504628,1.143300,-0.979666,0.750931,1.091962,0.644968,-0.810258,1.023059,-1.576939,1.783760,-1.997192,0.375575,-2.233522,0.824557,0.603287,-1.959785,0.645056,1.101540,-1.009328,2.402104,1.685323,0.386604,2.042387,-0.910862,-1.246986,-0.331123,0.027410,0.443654,-1.291464,0.301381,-0.655716,-0.372735,0.160851,-0.503871,1.123703,-0.950976,-0.205005,-0.654555,-1.551749,1.944104,-1.391381,0.745322,0.609470,0.234558,0.099144,-1.020924,2.415789,-0.856164,-1.336662,1.386519,2.437416,-0.077037,0.320741,-3.254244,-1.314506,0.722575,-1.563255,1.282448,-0.733094,-0.804640,0.264208,-0.284033,2,4,2,1,2,0.974990,0.0,0.0,1.0,0.997459,3,339.0,173,2.300125,0.0,0.996297
1,152,-1.555227,1.703838,-1.682477,1.143936,-1.363388,0.921123,1.209163,0.331891,-0.998299,1.015320,-1.242066,1.766697,-2.308836,0.442604,-2.749990,0.825489,0.596908,-1.176754,0.788825,0.962762,-1.042709,2.203321,1.695730,0.675732,2.172944,-1.027670,-1.245881,-0.080473,0.086483,0.430647,-1.400723,0.373409,-0.274544,0.677400,0.289406,-0.300185,0.725815,-0.903123,0.062829,-0.462003,-1.259109,2.258573,-1.285355,0.658545,0.672804,0.299276,-0.514287,-0.872554,2.333222,-0.890981,-1.186845,1.406470,2.004209,-0.169116,0.444842,-3.829849,-1.207255,0.239036,-1.438659,1.806309,-0.535842,-0.715147,-0.243989,-0.578661,2,1,2,1,0,0.987894,0.0,0.0,1.0,-9999.000000,1,98.0,54,1.598273,0.0,0.990280
2,714,-1.350453,1.907620,-1.563653,1.271580,-0.902961,0.582773,1.127565,0.447938,-1.022802,0.711931,-1.636709,1.885665,-2.092236,0.163758,-2.515830,0.828522,0.918868,-1.392947,1.453549,1.095837,-0.908404,2.470779,1.678719,-0.205788,2.162640,-1.063902,-1.219050,0.076042,0.341267,0.533246,-1.241235,0.506563,-0.416503,0.272600,0.325620,-0.808660,0.350219,-0.941883,-0.332984,-0.770360,-1.138200,2.354686,-1.409007,0.793536,0.674541,0.447000,-0.143163,-1.145194,1.947757,-0.832913,-1.443164,1.409840,2.089422,-0.462843,0.211531,-3.950086,-1.224400,0.911566,-1.581317,1.515879,-0.571897,-0.961354,-0.128131,-0.358940,2,2,2,1,1,0.991317,0.0,0.0,1.0,1.000000,2,235.0,109,1.368001,0.0,0.988229
3,5,-1.757556,2.429529,-1.438695,0.913385,0.016773,0.629686,1.096519,0.901125,-0.639822,0.610012,-2.448452,2.254705,-1.977030,0.244674,-2.132576,0.832347,0.174011,-1.245904,0.680392,1.120218,-0.973336,1.885519,1.641319,-0.427695,2.185217,-0.738151,-1.244538,-0.098398,0.586035,0.734197,-1.173107,0.733088,0.058661,0.360945,0.124782,-0.381303,0.442166,-1.142064,-0.315166,-0.944775,-0.986942,2.199443,-1.143928,1.136280,0.724500,0.491751,0.133391,-1.089635,1.786158,-1.264123,-1.255365,1.565839,1.854973,-0.334534,-0.233716,-3.841123,-1.307208,1.019935,-1.614235,0.013505,-0.506440,-1.273173,0.367237,-0.853831,1,0,1,0,0,0.980202,0.0,0.0,-9999.0,-9999.000000,1,92.0,19,2.023099,0.0,1.009288
4,720,-1.466042,1.946119,-1.857461,1.244526,-0.474639,0.481864,0.748999,0.766505,-1.170275,1.053901,-2.406875,1.865145,-2.132282,0.132549,-2.095424,0.823610,0.908864,-1.140471,0.430122,0.934647,-0.396494,2.205085,1.668042,0.144544,2.361988,-0.362801,-1.313798,0.404679,0.235088,0.616834,-0.828936,0.378164,-0.336752,0.584411,0.452267,-1.241397,0.634588,-0.308690,-1.749513,-1.347281,-1.498992,2.554470,-1.729132,0.219448,

In [85]:
outliers_finder=IsolationForest(random_state=42, verbose=0)

In [86]:
for col in tqdm(features_train_for_outliers.columns):
    outliers_finder.fit(features_train_for_outliers[[col]])
    anomalies_detected_train = outliers_finder.predict(features_train_for_outliers[[col]])
    anomalies_detected_test = outliers_finder.predict(features_test_for_outliers[[col]])
    train_data[f'{col}_outlier'] = anomalies_detected_train
    test_data[f'{col}_outlier'] = anomalies_detected_test

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [03:20<00:00,  2.48s/it]


In [87]:
train_data

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,target,entity_0017_cd_a,entity_0017_cd_b,entity_0017_cd_c,entity_0017_cd_d,repeat_user,entity_0006_rt_na,entity_0007_ct_na,entity_0009_rt_na,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0000_outlier,emb_0001_outlier,emb_0002_outlier,emb_0003_outlier,emb_0004_outlier,emb_0005_outlier,emb_0006_outlier,emb_0007_outlier,emb_0008_outlier,emb_0009_outlier,emb_0010_outlier,emb_0011_outlier,emb_0012_outlier,emb_0013_outlier,emb_0014_outlier,emb_0015_outlier,emb_0016_outlier,emb_0017_outlier,emb_0018_outlier,emb_0019_outlier,emb_0020_outlier,emb_0021_outlier,emb_0022_outlier,emb_0023_outlier,emb_0024_outlier,emb_0025_outlier,emb_0026_outlier,emb_0027_outlier,emb_0028_outlier,emb_0029_outlier,emb_0030_outlier,emb_0031_outlier,emb_0032_outlier,emb_0033_outlier,emb_0034_outlier,emb_0035_outlier,emb_0036_outlier,emb_0037_outlier,emb_0038_outlier,emb_0039_outlier,emb_0040_outlier,emb_0041_outlier,emb_0042_outlier,emb_0043_outlier,emb_0044_outlier,emb_0045_outlier,emb_0046_outlier,emb_0047_outlier,emb_0048_outlier,emb_0049_outlier,emb_0050_outlier,emb_0051_outlier,emb_0052_outlier,emb_0053_outlier,emb_0054_outlier,emb_0055_outlier,emb_0056_outlier,emb_0057_outlier,emb_0058_outlier,emb_0059_outlier,emb_0060_outlier,emb_0061_outlier,emb_0062_outlier,emb_0063_outlier,entity_0001_ct_outlier,entity_0002_ct_outlier,entity_0003_ct_outlier,entity_0004_ct_outlier,entity_0005_ct_outlier,entity_0006_rt_outlier,entity_0007_ct_outlier,entity_0008_ct_outlier,entity_0009_rt_outlier,entity_0010_rt_outlier,entity_0011_ct_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0014_rt_outlier,entity_0015_ct_outlier,entity_0016_rt_outlier
0,711,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,0.0,0.0,NaN,1.000000,3,446.0,207,1.166190,0.0,1.008388,1,0,0,0,1,0,0,1,1,0,1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,-1,-1,1,1,1,-1,1,1,1,1,1,-1,-1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,-1,1,1,-1,-1,-1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1
1,218,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,0.0,

In [88]:
test_data

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,entity_0017_cd_a,entity_0017_cd_b,entity_0017_cd_c,entity_0017_cd_d,repeat_user,entity_0006_rt_na,entity_0007_ct_na,entity_0009_rt_na,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0000_outlier,emb_0001_outlier,emb_0002_outlier,emb_0003_outlier,emb_0004_outlier,emb_0005_outlier,emb_0006_outlier,emb_0007_outlier,emb_0008_outlier,emb_0009_outlier,emb_0010_outlier,emb_0011_outlier,emb_0012_outlier,emb_0013_outlier,emb_0014_outlier,emb_0015_outlier,emb_0016_outlier,emb_0017_outlier,emb_0018_outlier,emb_0019_outlier,emb_0020_outlier,emb_0021_outlier,emb_0022_outlier,emb_0023_outlier,emb_0024_outlier,emb_0025_outlier,emb_0026_outlier,emb_0027_outlier,emb_0028_outlier,emb_0029_outlier,emb_0030_outlier,emb_0031_outlier,emb_0032_outlier,emb_0033_outlier,emb_0034_outlier,emb_0035_outlier,emb_0036_outlier,emb_0037_outlier,emb_0038_outlier,emb_0039_outlier,emb_0040_outlier,emb_0041_outlier,emb_0042_outlier,emb_0043_outlier,emb_0044_outlier,emb_0045_outlier,emb_0046_outlier,emb_0047_outlier,emb_0048_outlier,emb_0049_outlier,emb_0050_outlier,emb_0051_outlier,emb_0052_outlier,emb_0053_outlier,emb_0054_outlier,emb_0055_outlier,emb_0056_outlier,emb_0057_outlier,emb_0058_outlier,emb_0059_outlier,emb_0060_outlier,emb_0061_outlier,emb_0062_outlier,emb_0063_outlier,entity_0001_ct_outlier,entity_0002_ct_outlier,entity_0003_ct_outlier,entity_0004_ct_outlier,entity_0005_ct_outlier,entity_0006_rt_outlier,entity_0007_ct_outlier,entity_0008_ct_outlier,entity_0009_rt_outlier,entity_0010_rt_outlier,entity_0011_ct_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0014_rt_outlier,entity_0015_ct_outlier,entity_0016_rt_outlier
0,552,-1.786346,2.074665,-1.504628,1.143300,-0.979666,0.750931,1.091962,0.644968,-0.810258,1.023059,-1.576939,1.783760,-1.997192,0.375575,-2.233522,0.824557,0.603287,-1.959785,0.645056,1.101540,-1.009328,2.402104,1.685323,0.386604,2.042387,-0.910862,-1.246986,-0.331123,0.027410,0.443654,-1.291464,0.301381,-0.655716,-0.372735,0.160851,-0.503871,1.123703,-0.950976,-0.205005,-0.654555,-1.551749,1.944104,-1.391381,0.745322,0.609470,0.234558,0.099144,-1.020924,2.415789,-0.856164,-1.336662,1.386519,2.437416,-0.077037,0.320741,-3.254244,-1.314506,0.722575,-1.563255,1.282448,-0.733094,-0.804640,0.264208,-0.284033,2,4,2,1,2,0.974990,0.0,0.0,1.0,0.997459,3,339.0,173,2.300125,0.0,0.996297,0,0,0,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,-1,1,1,1,-1,1,1,1,1,1,1,1,-1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,-1,1,1,1,1,1,1
1,152,-1.555227,1.703838,-1.682477,1.143936,-1.363388,0.921123,1.209163,0.331891,-0.998299,1.015320,-1.242066,1.766697,-2.308836,0.442604,-2.749990,0.825489,0.596908,-1.176754,0.788825,0.962762,-1.042709,2.203321,1.695730,0.675732,2.172944,-1.027670,-1.245881,-0.080473,0.086483,0.430647,-1.400723,0.373409,-0.274544,0.677400,0.289406,-0.300185,0.725815,-0.903123,0.062829,-0.462003,-1.259109,2.258573,-1.285355,0.658545,0.672804,0.299276,-0.514287,-0.872554,2.333222,-0.890981,-1.186845,1.406470,2.004209,-0.169116,0.444842,-3.829849,-1.207255,0.239036,-1.438659,1.806309,-0.535842,-0.715147,-0.243989,-0.578661,2,1,2,1,0,0.987894,0.0,0.0,1.0,NaN,1,98.0,5

### feature selection

In [141]:
features_train_for_feature_selection = train_data.drop(columns='target').copy()
target_train_for_feature_selection = train_data['target']

In [142]:
features_train_for_feature_selection

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0007_ct,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0016_rt,entity_0017_cd_a,entity_0017_cd_b,entity_0017_cd_c,entity_0017_cd_d,repeat_user,entity_0006_rt_na,entity_0007_ct_na,entity_0009_rt_na,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0000_outlier,emb_0001_outlier,emb_0002_outlier,emb_0003_outlier,emb_0004_outlier,emb_0005_outlier,emb_0006_outlier,emb_0007_outlier,emb_0008_outlier,emb_0009_outlier,emb_0010_outlier,emb_0011_outlier,emb_0012_outlier,emb_0013_outlier,emb_0014_outlier,emb_0015_outlier,emb_0016_outlier,emb_0017_outlier,emb_0018_outlier,emb_0019_outlier,emb_0020_outlier,emb_0021_outlier,emb_0022_outlier,emb_0023_outlier,emb_0024_outlier,emb_0025_outlier,emb_0026_outlier,emb_0027_outlier,emb_0028_outlier,emb_0029_outlier,emb_0030_outlier,emb_0031_outlier,emb_0032_outlier,emb_0033_outlier,emb_0034_outlier,emb_0035_outlier,emb_0036_outlier,emb_0037_outlier,emb_0038_outlier,emb_0039_outlier,emb_0040_outlier,emb_0041_outlier,emb_0042_outlier,emb_0043_outlier,emb_0044_outlier,emb_0045_outlier,emb_0046_outlier,emb_0047_outlier,emb_0048_outlier,emb_0049_outlier,emb_0050_outlier,emb_0051_outlier,emb_0052_outlier,emb_0053_outlier,emb_0054_outlier,emb_0055_outlier,emb_0056_outlier,emb_0057_outlier,emb_0058_outlier,emb_0059_outlier,emb_0060_outlier,emb_0061_outlier,emb_0062_outlier,emb_0063_outlier,entity_0001_ct_outlier,entity_0002_ct_outlier,entity_0003_ct_outlier,entity_0004_ct_outlier,entity_0005_ct_outlier,entity_0006_rt_outlier,entity_0007_ct_outlier,entity_0008_ct_outlier,entity_0009_rt_outlier,entity_0010_rt_outlier,entity_0011_ct_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0014_rt_outlier,entity_0015_ct_outlier,entity_0016_rt_outlier
0,711,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,0.0,0.0,NaN,1.000000,3,446.0,207,1.166190,0.0,1.008388,0,0,0,1,0,0,1,1,0,1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,-1,-1,1,1,1,-1,1,1,1,1,1,-1,-1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,-1,1,1,-1,-1,-1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1
1,218,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,0.0,0.0,0.975

In [143]:
features_train_for_feature_selection.fillna(-9999, inplace=True)

In [144]:
# cfe = CountFrequencyEncoder(encoding_method='count')

In [145]:
# features_train_for_feature_selection = cfe.fit_transform(features_train_for_feature_selection)

In [146]:
# features_train_for_feature_selection = features_train_for_feature_selection.values

In [147]:
# target_train_for_feature_selection = target_train_for_feature_selection.values

In [148]:
# target_train_for_feature_selection.ravel()

In [149]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=10)

In [150]:
feature_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [151]:
feature_selector.fit(features_train_for_feature_selection.values, 
                     target_train_for_feature_selection.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	172
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	172
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	172
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	172
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	172
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	172
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	172
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	110
Tentative: 	15
Rejected: 	47
Iteration: 	9 / 100
Confirmed: 	110
Tentative: 	15
Rejected: 	47
Iteration: 	10 / 100
Confirmed: 	110
Tentative: 	15
Rejected: 	47
Iteration: 	11 / 100
Confirmed: 	110
Tentative: 	15
Rejected: 	47
Iteration: 	12 / 100
Confirmed: 	112
Tentative: 	13
Rejected: 	47
Iteration: 	13 / 100
Confirmed: 	112
Tentative: 	13
Rejected: 	47
Iteration: 	14 / 100
Confirmed: 	112
Tentative: 	13
Rejected: 	47
Iteration: 	15 / 100
Confirmed: 	112
Tentative: 	11
Rejected: 	49
Iteration: 	16 / 100
Confirmed: 	

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=6,
                                          n_estimators=250, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x1CC6CF3F8C8),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1CC6CF3F8C8, verbose=2)

In [152]:
feature_selector.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False, False,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True, False, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False,

In [153]:
len(feature_selector.support_)

172

In [154]:
need_cols = []
for col, status in zip(features_train_for_feature_selection.columns, feature_selector.support_):
    if status==True:
        need_cols.append(col)

In [155]:
# check ranking of features
feature_selector.ranking_

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, 60,  1,  1,  1,  1,  1,  1,  1,  1, 17,  1, 56, 20,  1,
       40,  1, 60,  1,  1,  1,  1,  1, 34,  1, 54, 15,  1, 34, 42,  8,  1,
       32, 51, 57, 23, 10, 14, 25, 31,  1,  1,  5, 47, 36, 18, 51, 45, 43,
        1, 27, 28, 50,  3, 46, 30,  4, 41, 55, 38, 26, 20, 37, 32, 24, 38,
       13,  1,  1,  1, 43,  1,  1,  1, 22,  1, 48,  7, 29,  2, 49,  8, 53,
        5, 11, 20,  1,  1,  1, 15,  1,  1, 60,  1,  1,  1, 12,  1,  1,  1,
        1, 58])

In [156]:
need_cols

['fact_dt',
 'emb_0000',
 'emb_0001',
 'emb_0002',
 'emb_0003',
 'emb_0004',
 'emb_0005',
 'emb_0006',
 'emb_0007',
 'emb_0008',
 'emb_0009',
 'emb_0010',
 'emb_0011',
 'emb_0012',
 'emb_0013',
 'emb_0014',
 'emb_0015',
 'emb_0016',
 'emb_0017',
 'emb_0018',
 'emb_0019',
 'emb_0020',
 'emb_0021',
 'emb_0022',
 'emb_0023',
 'emb_0024',
 'emb_0025',
 'emb_0026',
 'emb_0027',
 'emb_0028',
 'emb_0029',
 'emb_0030',
 'emb_0031',
 'emb_0032',
 'emb_0033',
 'emb_0034',
 'emb_0035',
 'emb_0036',
 'emb_0037',
 'emb_0038',
 'emb_0039',
 'emb_0040',
 'emb_0041',
 'emb_0042',
 'emb_0043',
 'emb_0044',
 'emb_0045',
 'emb_0046',
 'emb_0047',
 'emb_0048',
 'emb_0049',
 'emb_0050',
 'emb_0051',
 'emb_0052',
 'emb_0053',
 'emb_0054',
 'emb_0055',
 'emb_0056',
 'emb_0057',
 'emb_0058',
 'emb_0059',
 'emb_0060',
 'emb_0061',
 'emb_0062',
 'emb_0063',
 'entity_0001_ct',
 'entity_0002_ct',
 'entity_0003_ct',
 'entity_0004_ct',
 'entity_0005_ct',
 'entity_0006_rt',
 'entity_0008_ct',
 'entity_0009_rt',
 'en

### save features_train, features_test, target_train

In [159]:
features_train = train_data[need_cols]
target_train = train_data['target']

In [160]:
features_test = test_data[need_cols]

In [168]:
features_train

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0017_cd_a,entity_0017_cd_d,entity_0006_rt_na,entity_0009_rt_na,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0000_outlier,emb_0002_outlier,emb_0005_outlier,emb_0009_outlier,emb_0018_outlier,emb_0019_outlier,emb_0027_outlier,emb_0045_outlier,emb_0046_outlier,emb_0047_outlier,emb_0049_outlier,emb_0050_outlier,emb_0051_outlier,emb_0053_outlier,entity_0001_ct_outlier,entity_0002_ct_outlier,entity_0003_ct_outlier,entity_0005_ct_outlier,entity_0006_rt_outlier,entity_0008_ct_outlier,entity_0009_rt_outlier,entity_0010_rt_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0014_rt_outlier,entity_0015_ct_outlier
0,711,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,0.0,NaN,1.000000,3,446.0,207,1.166190,0.0,0,1,0,1,0,1,-1,1,1,1,1,1,1,-1,1,1,-1,1,-1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1
1,218,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,0.0,0.975221,0.996865,3,330.0,200,1.753914,0.0,0,1,0,0,0,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,-1,1,1,1,-1,1,1,-1,-1,1,1,1,1
2,462,-0.800151,2.204143,-0.514011,0.961015,-0.116245,0.218591,1.152742,0.955213,-1.075719,2.451778,-3.058330,2.097131,-2.285532,0.062348,-2.129199,0.829396,0.642969,-1.849109,-0.553088,0.575978,-0.848553,1.657650,1.695016,0.018969,2.155971,-0.469931,-0.967494,-0.895899,-0.364366,0.944774,-0.673376,0.791378,-0.214279,0.588130,0.450273,-0.892531,0.754762,-0.814439,-0.444689,-0.521155,-0.792310,2.015573,-1.363314,0.924737,0.155714,-0.670974,-0.793958,-0.905191,2.167285,-1.523553,-0.857246,1.325879,1.684667,0.385727,0.096711,-2.704222,-1.160303,0.934533,-1.503963,-0.016609,-0.465335,-1.034321,0.653358,-0.727603,4,0,4,0,2,0.987922,0.0,NaN,0.781602,2,191.0,82,1.548657,0.0,0,0,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,1,1,-1,-1,1,1,1,1
3,13,-1.543821,2.044538,-1.288806,0.958519,-0.485421,0.534662,0.498838,0.913363,-1.150698,0.682353,-2.342354,1.619433,-2.008181,0.317489,-2.045199,0.819717,0.359040,-1.013714,0.515654,0.962981,-0.895936,2.444055,1.650715,0.488509,2.050253,-0.513660,-1.617719,-0.198799,0.332519,0.498607,-1.393829,0.163533,-0.21

In [169]:
features_test

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0008_ct,entity_0009_rt,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0017_cd_a,entity_0017_cd_d,entity_0006_rt_na,entity_0009_rt_na,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0000_outlier,emb_0002_outlier,emb_0005_outlier,emb_0009_outlier,emb_0018_outlier,emb_0019_outlier,emb_0027_outlier,emb_0045_outlier,emb_0046_outlier,emb_0047_outlier,emb_0049_outlier,emb_0050_outlier,emb_0051_outlier,emb_0053_outlier,entity_0001_ct_outlier,entity_0002_ct_outlier,entity_0003_ct_outlier,entity_0005_ct_outlier,entity_0006_rt_outlier,entity_0008_ct_outlier,entity_0009_rt_outlier,entity_0010_rt_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0014_rt_outlier,entity_0015_ct_outlier
0,552,-1.786346,2.074665,-1.504628,1.143300,-0.979666,0.750931,1.091962,0.644968,-0.810258,1.023059,-1.576939,1.783760,-1.997192,0.375575,-2.233522,0.824557,0.603287,-1.959785,0.645056,1.101540,-1.009328,2.402104,1.685323,0.386604,2.042387,-0.910862,-1.246986,-0.331123,0.027410,0.443654,-1.291464,0.301381,-0.655716,-0.372735,0.160851,-0.503871,1.123703,-0.950976,-0.205005,-0.654555,-1.551749,1.944104,-1.391381,0.745322,0.609470,0.234558,0.099144,-1.020924,2.415789,-0.856164,-1.336662,1.386519,2.437416,-0.077037,0.320741,-3.254244,-1.314506,0.722575,-1.563255,1.282448,-0.733094,-0.804640,0.264208,-0.284033,2,4,2,1,2,0.974990,0.0,1.0,0.997459,3,339.0,173,2.300125,0.0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,-1,1,1,1,1
1,152,-1.555227,1.703838,-1.682477,1.143936,-1.363388,0.921123,1.209163,0.331891,-0.998299,1.015320,-1.242066,1.766697,-2.308836,0.442604,-2.749990,0.825489,0.596908,-1.176754,0.788825,0.962762,-1.042709,2.203321,1.695730,0.675732,2.172944,-1.027670,-1.245881,-0.080473,0.086483,0.430647,-1.400723,0.373409,-0.274544,0.677400,0.289406,-0.300185,0.725815,-0.903123,0.062829,-0.462003,-1.259109,2.258573,-1.285355,0.658545,0.672804,0.299276,-0.514287,-0.872554,2.333222,-0.890981,-1.186845,1.406470,2.004209,-0.169116,0.444842,-3.829849,-1.207255,0.239036,-1.438659,1.806309,-0.535842,-0.715147,-0.243989,-0.578661,2,1,2,1,0,0.987894,0.0,1.0,NaN,1,98.0,54,1.598273,0.0,1,0,0,0,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,-1,1,1,1,1
2,714,-1.350453,1.907620,-1.563653,1.271580,-0.902961,0.582773,1.127565,0.447938,-1.022802,0.711931,-1.636709,1.885665,-2.092236,0.163758,-2.515830,0.828522,0.918868,-1.392947,1.453549,1.095837,-0.908404,2.470779,1.678719,-0.205788,2.162640,-1.063902,-1.219050,0.076042,0.341267,0.533246,-1.241235,0.506563,-0.416503,0.272600,0.325620,-0.808660,0.350219,-0.941883,-0.332984,-0.770360,-1.138200,2.354686,-1.409007,0.793536,0.674541,0.447000,-0.143163,-1.145194,1.947757,-0.832913,-1.443164,1.409840,2.089422,-0.462843,0.211531,-3.950086,-1.224400,0.911566,-1.581317,1.515879,-0.571897,-0.961354,-0.128131,-0.358940,2,2,2,1,1,0.991317,0.0,1.0,1.000000,2,235.0,109,1.368001,0.0,0,1,0,0,0,1,-1,1,1,1,1,-1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,5,-1.757556,2.429529,-1.438695,0.913385,0.016773,0.629686,1.096519,0.901125,-0.639822,0.610012,-2.448452,2.254705,-1.977030,0.244674,-2.132576,0.832347,0.174011,-1.245904,0.680392,1.120218,-0.973336,1.885519,1.641319,-0.427695,2.185217,-0.738151,-1.244538,-0.098398,0.586035,0.734197,-1.173107,0.733088,0.058661,0.360945,0.124782,-0.381303,0.442

In [170]:
features_train.to_pickle(f'{PATH_TO_PREPARED_DATA}/features_train.pickle')
features_test.to_pickle(f'{PATH_TO_PREPARED_DATA}/features_test.pickle')
target_train.to_pickle(f'{PATH_TO_PREPARED_DATA}/target_train.pickle')

# Models

In [28]:
best_model = joblib.load(f'{PATH_TO_ARTEFACTS}/model.pickle')

In [33]:
best_model.get_params()

{'memory': None,
 'steps': [('imputer', None),
  ('scaler', None),
  ('model', <catboost.core.CatBoostClassifier at 0x1fa02767fc8>)],
 'verbose': False,
 'imputer': None,
 'scaler': None,
 'model': <catboost.core.CatBoostClassifier at 0x1fa02767fc8>,
 'model__l2_leaf_reg': 10,
 'model__verbose': 0,
 'model__max_depth': 10,
 'model__n_estimators': 1000,
 'model__random_state': 42,
 'model__min_data_in_leaf': 1}

## read prepared files

In [20]:
features_train = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/features_train.pickle')
features_test = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/features_test.pickle')
target_train = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/target_train.pickle')

In [21]:
features_train

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0008_ct,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0017_cd_a,entity_0017_cd_d,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0045_outlier,entity_0006_rt_outlier,entity_0008_ct_outlier,entity_0010_rt_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0015_ct_outlier
0,711,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,NaN,1.000000,3,446.0,207,1.166190,NaN,0,1,0,1,-1,1,1,1,1,1,1,1
1,218,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,NaN,0.996865,3,330.0,200,1.753914,NaN,0,1,0,1,1,1,1,1,-1,1,1,1
2,462,-0.800151,2.204143,-0.514011,0.961015,-0.116245,0.218591,1.152742,0.955213,-1.075719,2.451778,-3.058330,2.097131,-2.285532,0.062348,-2.129199,0.829396,0.642969,-1.849109,-0.553088,0.575978,-0.848553,1.657650,1.695016,0.018969,2.155971,-0.469931,-0.967494,-0.895899,-0.364366,0.944774,-0.673376,0.791378,-0.214279,0.588130,0.450273,-0.892531,0.754762,-0.814439,-0.444689,-0.521155,-0.792310,2.015573,-1.363314,0.924737,0.155714,-0.670974,-0.793958,-0.905191,2.167285,-1.523553,-0.857246,1.325879,1.684667,0.385727,0.096711,-2.704222,-1.160303,0.934533,-1.503963,-0.016609,-0.465335,-1.034321,0.653358,-0.727603,4,0,4,0,2,0.987922,NaN,0.781602,2,191.0,82,1.548657,NaN,0,0,0,1,1,-1,1,1,-1,1,1,1
3,13,-1.543821,2.044538,-1.288806,0.958519,-0.485421,0.534662,0.498838,0.913363,-1.150698,0.682353,-2.342354,1.619433,-2.008181,0.317489,-2.045199,0.819717,0.359040,-1.013714,0.515654,0.962981,-0.895936,2.444055,1.650715,0.488509,2.050253,-0.513660,-1.617719,-0.198799,0.332519,0.498607,-1.393829,0.163533,-0.217686,0.988170,-0.014833,-1.646648,1.749996,0.100091,-0.908938,-1.034925,-1.161342,2.306280,-1.087395,0.695601,1.123252,0.504989,-0.053581,-1.101684,2.047417,-0.872976,-1.436043,1.493698,1.900455,-0.137229,-0.173302,-3.522463,-1.296566,0.875170,-1.407083,0.554178,-0.708667,-1.267630,0.610148,-0.363370,4,4,4,0,2,0.990692,NaN,0.000000,4,737.0,328,1.385834,NaN,1,0,0,1,-1,1,1,1,1,1,1,1
4,371,-1.564026,2.291450,-1.680573,1.553684,-0.931364,0.613911,1.167507,0.668621,-0.892333,0.613059,-1.565685,1.777778,-1.983498,0.378075,-2.488828,0.823546,0.677798,-1.135342,1.158549,1.225101,

In [22]:
features_test

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0008_ct,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0017_cd_a,entity_0017_cd_d,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0045_outlier,entity_0006_rt_outlier,entity_0008_ct_outlier,entity_0010_rt_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0015_ct_outlier
0,552,-1.786346,2.074665,-1.504628,1.143300,-0.979666,0.750931,1.091962,0.644968,-0.810258,1.023059,-1.576939,1.783760,-1.997192,0.375575,-2.233522,0.824557,0.603287,-1.959785,0.645056,1.101540,-1.009328,2.402104,1.685323,0.386604,2.042387,-0.910862,-1.246986,-0.331123,0.027410,0.443654,-1.291464,0.301381,-0.655716,-0.372735,0.160851,-0.503871,1.123703,-0.950976,-0.205005,-0.654555,-1.551749,1.944104,-1.391381,0.745322,0.609470,0.234558,0.099144,-1.020924,2.415789,-0.856164,-1.336662,1.386519,2.437416,-0.077037,0.320741,-3.254244,-1.314506,0.722575,-1.563255,1.282448,-0.733094,-0.804640,0.264208,-0.284033,2,4,2,1,2,0.974990,NaN,0.997459,3,339.0,173,2.300125,NaN,0,1,0,1,1,1,1,1,-1,1,1,1
1,152,-1.555227,1.703838,-1.682477,1.143936,-1.363388,0.921123,1.209163,0.331891,-0.998299,1.015320,-1.242066,1.766697,-2.308836,0.442604,-2.749990,0.825489,0.596908,-1.176754,0.788825,0.962762,-1.042709,2.203321,1.695730,0.675732,2.172944,-1.027670,-1.245881,-0.080473,0.086483,0.430647,-1.400723,0.373409,-0.274544,0.677400,0.289406,-0.300185,0.725815,-0.903123,0.062829,-0.462003,-1.259109,2.258573,-1.285355,0.658545,0.672804,0.299276,-0.514287,-0.872554,2.333222,-0.890981,-1.186845,1.406470,2.004209,-0.169116,0.444842,-3.829849,-1.207255,0.239036,-1.438659,1.806309,-0.535842,-0.715147,-0.243989,-0.578661,2,1,2,1,0,0.987894,NaN,NaN,1,98.0,54,1.598273,NaN,1,0,1,1,-1,1,1,1,-1,1,1,1
2,714,-1.350453,1.907620,-1.563653,1.271580,-0.902961,0.582773,1.127565,0.447938,-1.022802,0.711931,-1.636709,1.885665,-2.092236,0.163758,-2.515830,0.828522,0.918868,-1.392947,1.453549,1.095837,-0.908404,2.470779,1.678719,-0.205788,2.162640,-1.063902,-1.219050,0.076042,0.341267,0.533246,-1.241235,0.506563,-0.416503,0.272600,0.325620,-0.808660,0.350219,-0.941883,-0.332984,-0.770360,-1.138200,2.354686,-1.409007,0.793536,0.674541,0.447000,-0.143163,-1.145194,1.947757,-0.832913,-1.443164,1.409840,2.089422,-0.462843,0.211531,-3.950086,-1.224400,0.911566,-1.581317,1.515879,-0.571897,-0.961354,-0.128131,-0.358940,2,2,2,1,1,0.991317,NaN,1.000000,2,235.0,109,1.368001,NaN,0,1,0,1,-1,1,1,1,1,1,1,1
3,5,-1.757556,2.429529,-1.438695,0.913385,0.016773,0.629686,1.096519,0.901125,-0.639822,0.610012,-2.448452,2.254705,-1.977030,0.244674,-2.132576,0.832347,0.174011,-1.245904,0.680392,1.120218,-0.973336,1.885519,1.641319,-0.427695,2.185217,-0.738151,-1.244538,-0.098398,0.586035,0.734197,-1.173107,0.733088,0.058661,0.360945,0.124782,-0.381303,0.442166,-1.142064,-0.315166,-0.944775,-0.986942,2.199443,-1.143928,1.136280,0.724500,0.491751,0.133391,-1.089635,1.786158,-1.264123,-1.255365,1.565839,1.854973,-0.334534,-0.233716,-3.841123,-1.307208,1.019935,-1.614235,0.013505,-0.506440,-1.273173,0.367237,-0.853831,1,0,1,0,0,0.980202,NaN,NaN,1,92.0,19,2.023099,NaN,1,0,1,1,-1,1,1,1,-1,1,1,1
4,720,-1.466042,1.946119,-1.857461,1.244526,-0.474639,0.481864,0.748999,0.766505,-1.170275,1.053901,-2.406875,1.865145,-2.132282,0.132549,-2.095424,0.823610,0.908864,-1.140471,0.430122,0.934647,-0.396494,2.205085,

## baseline model

In [23]:
pipeline_stages = []

# pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))

# pipeline_stages.append(("encoder", CountFrequencyEncoder(encoding_method='count')))
# pipeline_stages.append(("encoder", MeanEncoder()))

# pipeline_stages.append(("imputer", SimpleImputer(strategy='median')))

# pipeline_stages.append(("scaler", MinMaxScaler()))

# pipeline_stages.append(("poly_features", PolynomialFeatures(degree=2, interaction_only=True)))

pipeline_stages.append(("model", CatBoostClassifier(verbose=0)))
# pipeline_stages.append(("model", LogisticRegression()))
# pipeline_stages.append(("model", IsolationForest()))


# Собираем все шаги в пайплайн
pipeline = Pipeline(pipeline_stages)

In [24]:
tscv = StratifiedKFold()

In [25]:
for train_idx, valid_idx in tscv.split(features_train, target_train):
    # разбиваем по фолдам
    X_train, X_test = features_train.iloc[train_idx], features_train.iloc[valid_idx]
    y_train, y_test = target_train.iloc[train_idx], target_train.iloc[valid_idx]
    # обучаем модель и считаем метрики
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)
    print(roc_auc_score(y_test, y_pred[:, 1]))

NameError: name 'calibrated_clf' is not defined

## find best model

In [89]:
pipeline_stages = []

pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))
pipeline_stages.append(("scaler", StandardScaler()))
pipeline_stages.append(("model", CatBoostClassifier(verbose=0)))

pipeline = Pipeline(pipeline_stages)

In [32]:
param_grid = [
    {
        "imputer": [SimpleImputer(strategy='median'), ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [CatBoostClassifier(random_state=42, verbose=0)],
        "model__n_estimators": [500, 1000],
        "model__max_depth": [6, 8, 10, 12],
        'model__loss_function': ['Logloss', 'CrossEntropy'],
        "model__l2_leaf_reg": [3, 4, 5], 
        "model__min_data_in_leaf": [1, 3, 5],
        "scaler": [None]
    },
    {
        "imputer": [SimpleImputer(strategy='median'), ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [LogisticRegression(random_state=42)],
        "model__C": [0.001, 0.01, 0.1, 1, 10, 100],
        "model__penalty":['none', 'elasticnet', 'l1', 'l2'],
        "model__solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        "scaler": [StandardScaler()]
    },
]

In [33]:
tscv = StratifiedKFold(n_splits=5)

In [34]:
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring="roc_auc", cv=tscv, verbose=2)

In [35]:
grid.fit(features_train, target_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=6, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=   1.3s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=6, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=   1.2s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=6, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=   1.2s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0

[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=8, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=   3.2s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=8, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=   3.2s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=8, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=   3.4s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=L

[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=10, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=  12.6s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=10, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=  12.8s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=10, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=  13.0s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_functio

[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=12, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=  47.1s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=12, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=  53.1s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_function=Logloss, model__max_depth=12, model__min_data_in_leaf=1, model__n_estimators=500, scaler=None; total time=  39.2s
[CV] END imputer=SimpleImputer(strategy='median'), model=<catboost.core.CatBoostClassifier object at 0x000001FE04CADA88>, model__l2_leaf_reg=3, model__loss_functio

KeyboardInterrupt: 

In [133]:
grid.best_estimator_

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', None),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000001C86612EB88>)])

In [134]:
grid.best_score_

0.8227277186754941

In [135]:
grid.cv_results_

{'mean_fit_time': array([ 19.11948934,  25.13344836,  18.03744574,  24.04043727,
          7.16799397,  13.06757479,  11.44378529,  21.87818761,
         66.54465032, 126.06038485,   6.9621953 ,  13.25820951,
         10.71092715,  20.88135905,  64.5014123 , 127.96483912,
          1.85300155,   1.88480868,   1.85302224,   1.88884525,
          1.15260811,   1.15080085,   1.1550096 ,   1.14460125]),
 'std_fit_time': array([0.17109743, 0.42735009, 0.22588032, 0.24513569, 0.36806234,
        0.82862448, 0.21467402, 0.67217387, 1.49379918, 0.55961293,
        0.12646998, 0.11436095, 0.10119494, 0.28086133, 0.3164754 ,
        2.08384948, 0.02819879, 0.03680501, 0.0205568 , 0.03875085,
        0.02280459, 0.01913632, 0.01588931, 0.0104794 ]),
 'mean_score_time': array([0.15339179, 0.19319344, 0.14740767, 0.19101315, 0.02801223,
        0.03001795, 0.02921448, 0.03040757, 0.04221611, 0.05402122,
        0.03020878, 0.03040881, 0.03060789, 0.0316009 , 0.04279499,
        0.06456175, 0.032197

In [ ]:
calibrated_clf = CalibratedClassifierCV(base_clf, cv=3)

In [ ]:
# predict probabilities
probs = model.predic_proba(testX)[:,1]
# reliability diagram
fop, mpv = calibration_curve(testy, probs, n_bins=10)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

# Make submission

In [136]:
pipeline = grid.best_estimator_

In [137]:
pipeline.fit(features_train, target_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', None),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000001C86612EB88>)])

In [138]:
y_pred = pipeline.predict_proba(features_test)

In [139]:
y_pred = y_pred[:, 1]

In [140]:
len(y_pred), sum(y_pred)

(26824, 23336.148694900312)

In [141]:
submit_data = pd.DataFrame(data=y_pred, columns=['Predicted'])

In [142]:
submit_data.to_csv(f'{PATH_TO_PREPARED_DATA}/submit.csv', index_label='Id', sep=",")

In [143]:
submit_data[submit_data['Predicted']==0]

,Predicted
